### **1.安装 IndexTTS-2**

In [ ]:
%cd /content
# !rm -rf /content/index-tts
!git lfs install
!git clone https://github.com/flashclub/index-tts.git
%cd index-tts
!git lfs pull
!wget https://raw.githubusercontent.com/NeuralFalconYT/Useful-Function/refs/heads/main/hf_downloader.py
!pip install uv --quiet
!uv sync --all-extras
from IPython.display import clear_output
clear_output()

### **2.下载模型**

In [ ]:
%cd /content/index-tts
from hf_downloader import download_model
from IPython.display import clear_output

model_path = download_model(
    "IndexTeam/IndexTTS-2",
    download_folder="./checkpoints",
    redownload=False
)
clear_output()
print("✅ 模型保存至:", model_path)

### **3.配置环境变量**

> **重要提示**: 使用新的 `/api/synthesize_with_storage` 接口需要配置以下环境变量:
> - R2 配置: `R2_ACCOUNT_ID`, `R2_ACCESS_KEY_ID`, `R2_SECRET_ACCESS_KEY`, `R2_BUCKET_NAME`, `R2_PUBLIC_URL`
> - Supabase 配置: `SUPABASE_URL`, `SUPABASE_KEY`
> 
> 请在下方单元格中填入你的配置信息。

In [ ]:
# 基础配置
%env NGROK_AUTHTOKEN=36F1N7edSKgZSUjLOpy4dMovTIf_NStRRaxSJ3hcb3tCermQ

# Cloudflare R2 配置 - 请替换为你的实际配置
%env R2_ACCOUNT_ID=your-account-id
%env R2_ACCESS_KEY_ID=your-access-key-id
%env R2_SECRET_ACCESS_KEY=your-secret-access-key
%env R2_BUCKET_NAME=your-bucket-name
%env R2_PUBLIC_URL=https://your-bucket.r2.dev

# Supabase 配置 - 请替换为你的实际配置
%env SUPABASE_URL=https://your-project.supabase.co
%env SUPABASE_KEY=your-service-role-key

print("✅ 环境变量配置完成")
print("\n提示: 如果不需要使用云存储功能，可以使用原有的 /api/synthesize 接口")

### **4.启动 API 服务**

服务启动后会提供两个接口:

1. **`/api/synthesize`** - 原有接口,生成音频并保存到本地 `/content/drive/MyDrive/Index-TTS/outputs`
2. **`/api/synthesize_with_storage`** - 新接口,生成音频后上传到 R2 并存储到 Supabase,会自动删除本地文件

**注意**: 如果未配置 R2/Supabase 环境变量,新接口将不可用,但不影响原有接口的使用。

In [ ]:
%cd /content/index-tts
!uv run index_tts_api.py --model_dir "/content/index-tts/checkpoints/IndexTTS-2"

### **5.测试新接口 (可选)**

在另一个终端或 notebook 中测试新接口:

In [ ]:
# 测试 /api/synthesize_with_storage 接口
import requests

# 请替换为你的 ngrok URL
API_URL = "https://your-ngrok-url.ngrok.io/api/synthesize_with_storage"

response = requests.post(
    API_URL,
    data={
        "voice_path": "/content/index-tts/examples/voices/female.wav",  # 替换为实际的音色文件路径
        "text": "这是一段测试文本，用于验证新的API接口是否正常工作。"
    }
)

if response.status_code == 200:
    result = response.json()
    print("✅ 合成成功!")
    print(f"R2 URL: {result['r2_url']}")
    print(f"Supabase 记录: {result['supabase_record']}")
else:
    print(f"❌ 请求失败: {response.status_code}")
    print(response.text)